In [1]:
# -*- coding: utf-8 -*-
import array
import random
import json

import numpy as np
import matplotlib.pyplot as plt

from math import sqrt

from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools

In [9]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMax)
toolbox = base.Toolbox()

BOUND_LOW, BOUND_UP = 0.0, 1.0

NDIM = 30

def uniform(low, up, size=None):
    try:
        print("a")
        return [random.uniform(a, b) for a, b in zip(low, up)]
    except TypeError:
        print("b")
        return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]


toolbox.register("attr_float", uniform, BOUND_LOW, BOUND_UP, NDIM)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", benchmarks.zdt1)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0, indpb=1.0/NDIM)
toolbox.register("select", tools.selNSGA2)

def main(seed=None):
    random.seed(seed)

    NGEN = 250
    MU = 100
    CXPB = 0.9

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)
    
    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "min", "max"
    
    pop = toolbox.population(n=MU)
    pop_ini = pop[:]
    
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    pop = toolbox.select(pop, len(pop))
    
    record = stats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print(logbook.stream)

    for gen in range(1, NGEN):

        offspring = tools.selTournamentDCD(pop, len(pop))
        offspring = [toolbox.clone(ind) for ind in offspring]
        for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
            if random.random() <= CXPB:
                toolbox.mate(ind1, ind2)
            
            toolbox.mutate(ind1)
            toolbox.mutate(ind2)
            del ind1.fitness.values, ind2.fitness.values
        
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop = toolbox.select(pop + offspring, MU)
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        print(logbook.stream)

    print("Final population hypervolume is %f" % hypervolume(pop, [11.0, 11.0]))

    return pop, pop_ini, logbook
        
if __name__ == "__main__":
    pop, pop_ini, stats = main(64)

    fitnesses_ini = np.array([list(pop_ini[i].fitness.values) for i in range(len(pop_ini))])
    fitnesses = np.array([list(pop[i].fitness.values) for i in range(len(pop))])

    plt.plot(fitnesses_ini[:,0], fitnesses_ini[:,1], "b.", label="Initial")
    plt.plot(fitnesses[:,0], fitnesses[:,1], "r.", label="Optimized" )
    plt.legend(loc="upper right")
    plt.title("fitnesses")
    plt.xlabel("f1")
    plt.ylabel("f2")
    plt.grid(True)
    #plt.savefig("fitnesses.png", dpi=300)

/Users/puca/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/puca/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	evals	min                      	max                      
0  	100  	[ 0.00776997  2.62064288]	[ 0.99151989  5.4725374 ]
1  	100  	[ 0.00776997  2.62064288]	[ 0.9747426   5.78429803]
2  	100  	[ 0.0066606   2.62064288]	[ 0.98169821  5.71095328]
3  	100  	[ 0.00453496  2.30171896]	[ 0.98169821  5.44634658]
4  	100  	[  1.86879579e-03   2.30171896e+00]	[ 0.90650838  5.21355838]
5  	100  	[  1.23896591e-03   2.10149660e+00]	[ 0.90660299  5.21355838]
6  	100  	[  8.86062841e-04   1.92992834e+00]	[ 0.90517986  4.97438952]
7  	100  	[  6.73784153e-04   1.92920779e+00]	[ 0.90517986  4.5247554 ]
8  	100  	[  6.73784153e-04   1.64930097e+00]	[ 0.90517986  4.23140595]
9  	100  	[  6.73784153e-04   1.58967349e+00]	[ 0.88858703  3.97908873]
10 	100  	[  6.52539372e-05   1.53436394e+00]	[ 0.84033775  3.77519649]
11 	100  	[  6.52539372e-05   1.52616522e+00]	[ 0.84033775  3.71077393]
12 	100  	[  6.52539372e-05   1.16352217e+00]	[ 0.91270576  3.64949076]
13 	100  	[  5.49007925e-06   1.16352217e+

In [19]:
[0.0]+[-5.0]*2


[0.0, -5.0, -5.0]